# Embedding Bag Test
---

Experimenting applying an embedding bag (embedding layer + average of all embedding vectors) on the categorical features of a time series dataframe.

## Import the necessary packages

In [ ]:
# import dask.dataframe as dd                # Dask to handle big data in dataframes
# from dask.distributed import Client        # Dask scheduler
import pandas as pd                        # Pandas to load the data initially
import numpy as np                         # Mathematical operations package, allowing also for missing values representation
import torch                               # PyTorch for tensor and deep learning operations
import data_utils as du                    # Data science and machine learning relevant methods

In [ ]:
du.set_pandas_library('pandas')

In [ ]:
import pixiedust                           # Debugging in Jupyter Notebook cells

## Initialize variables

Data that we'll be using:

In [ ]:
data_df = pd.DataFrame([[103, 0, 'dog'], 
                        [103, 0, 'cat'], 
                        [103, 0, 'cat'],
                        [103, 1, 'horse'],
                        [103, 2, 'dog'],
                        [104, 0, 'bunny'],
                        [104, 1, np.nan],
                        [104, 2, 'bunny'],
                        [104, 3, np.nan],
                        [105, 0, 'horse'],
                        [105, 0, 'dog'],
                        [105, 0, 'dog'],
                        [105, 0, 'cat'],
                        [105, 0, 'bunny'],
                        [105, 0, 'horse'],
                        [105, 1, 'horse'],
                        [105, 1, 'bunny'],
                        [105, 1, 'dog'],
                        [105, 1, np.nan],
                        [105, 1, 'horse'],
                        [105, 1, 'horse'],
                        [105, 1, 'horse'],
                        [105, 1, 'horse'],
                        [105, 1, 'horse'],
                        [105, 1, 'horse']], columns=['id', 'ts', 'Var0'])
# Only use the line of code bellow if you want to test on Dask
# data_df = dd.from_pandas(data_df, npartitions=2)
# If using Pandas, uncomment the line of code bellow and comment the next one, which uses Dask
data_df
# data_df.compute()

In [ ]:
data_df.to_csv('dummy_data/embedding_bag_data_df.csv')

In [ ]:
data_df.Var0.str.replace('nn', 'AI')

In [ ]:
data_df.Var0.replace('nn', 'AI')

Embedding matrix used in the embedding layer:

In [ ]:
embed_mtx = torch.FloatTensor([[0, 0, 0],
                               [-1, 0, 1],
                               [0, 1, -1],
                               [1, 1, 0],
                               [1, -1, 1]])
embed_mtx

Simple embedding layer:

In [ ]:
simple_embed_layer = torch.nn.Embedding.from_pretrained(embed_mtx)
simple_embed_layer

Embedding layer + average operation (bagging):

In [ ]:
bag_embed_layer = torch.nn.EmbeddingBag.from_pretrained(embed_mtx)
bag_embed_layer

## One hot encode categories

In [ ]:
data_df_ohe = du.data_processing.one_hot_encoding_dataframe(data_df, columns='Var0', join_rows=False,
                                                            join_by=['id', 'ts'])
data_df_ohe

In [ ]:
data_df_ohe.loc[20, 'Var0_horse'] = np.nan
data_df_ohe.loc[18, 'Var0_cat'] = np.nan
data_df_ohe

In [ ]:
bool_cols = du.search_explore.list_boolean_columns(data_df_ohe)
bool_cols

In [ ]:
data_df_ohe = du.embedding.join_repeated_rows(data_df_ohe, id_columns=['id', 'ts'])
data_df_ohe

In [ ]:
data_df_ohe.to_csv('dummy_data/embedding_bag_data_df_ohe.csv')

In [ ]:
data_df_ohe.dtypes

In [ ]:
data_tnsr = du.padding.dataframe_to_padded_tensor(data_df_ohe, id_column='id')
data_tnsr

In [ ]:
ohe_feat = [du.search_explore.find_col_idx(data_df_ohe, feat) for feat in ['Var0_bunny', 'Var0_cat', 'Var0_dog', 'Var0_horse']]
ohe_feat

In [ ]:
ohe_data = data_tnsr[:, :, 2:].int()
ohe_data

In [ ]:
encod_tnsr = ohe_data * torch.tensor(list(range(1, ohe_data.shape[-1]+1)))
encod_tnsr

In [ ]:
encod_tnsr = encod_tnsr.view(-1, 4)
encod_tnsr

In [ ]:
tmp_data = data_tnsr[:, :, :2].view(-1, 2).long()
encod_tnsr = encod_tnsr[tmp_data[:, 0] != 999999]
tmp_data = tmp_data[tmp_data[:, 0] != 999999]
tmp_data

In [ ]:
tmp_data.shape

In [ ]:
encod_tnsr.shape

In [ ]:
torch.cat((tmp_data, encod_tnsr), axis=1)

In [ ]:
data_df_encod = pd.DataFrame(torch.cat((tmp_data, encod_tnsr), axis=1).numpy(), columns=data_df_ohe.columns)
data_df_encod

In [ ]:
data_df_encod.to_csv('dummy_data/embedding_bag_data_df_encod.csv')

In [ ]:
encod_tnsr.sum(axis=len(encod_tnsr.shape)-1)

In [ ]:
all_zero_row = (encod_tnsr.sum(axis=len(encod_tnsr.shape)-1) == 0)
all_zero_row

In [ ]:
all_zero_row.t()

In [ ]:
# all_zero_row = all_zero_row.repeat(encod_tnsr.shape[-1], 1).t()
# all_zero_row

In [ ]:
# 3D
torch.zeros(encod_tnsr.shape[0], encod_tnsr.shape[1], encod_tnsr.shape[2]-1).bool()
# 2D
# torch.zeros(encod_tnsr.shape[0], encod_tnsr.shape[1]-1).bool()

In [ ]:
# 3D
all_zero_row.view(encod_tnsr.shape[0], encod_tnsr.shape[1], -1)
# 2D
# all_zero_row.view(encod_tnsr.shape[0], -1)

In [ ]:
# 3D
all_zero_row = torch.cat([all_zero_row.view(encod_tnsr.shape[0], encod_tnsr.shape[1], -1), 
                          torch.zeros(encod_tnsr.shape[0], encod_tnsr.shape[1], encod_tnsr.shape[2]-1).bool()], axis=2)
# 2D
# all_zero_row = torch.cat([all_zero_row.view(encod_tnsr.shape[0], -1), 
#                           torch.zeros(encod_tnsr.shape[0], encod_tnsr.shape[1]-1).bool()], axis=1)
all_zero_row

In [ ]:
zero_val = (encod_tnsr != 0)
zero_val

In [ ]:
all_zero_row | zero_val

In [ ]:
encod_list = encod_tnsr[all_zero_row | zero_val]
encod_list

In [ ]:
encod_tnsr

In [ ]:
offset_list = (all_zero_row | zero_val).sum(axis=len(encod_tnsr.shape)-1).view(-1).cumsum(0)
offset_list + 1
offset_list = torch.cat([torch.zeros(1).long(), offset_list])[:-1]
offset_list

In [ ]:
du.embedding.prepare_embed_bag(data_tnsr, features=ohe_feat)

In [ ]:
embedding_values = bag_embed_layer(encod_list, offset_list)
embedding_values

In [ ]:
embed_mtx

## Enumerate categories

In [ ]:
data_df.Var0, enum_dict = du.embedding.enum_categorical_feature(data_df, 'Var0', nan_value=0, forbidden_digit=0)
# If using Pandas, uncomment the line of code bellow and comment the next one, which uses Dask
data_df
# data_df.compute()

In [ ]:
enum_dict

In [ ]:
# If using Pandas, uncomment the line of code bellow and comment the next one, which uses Dask
data = torch.tensor(data_df.values)
# data = torch.tensor(data_df.compute().values)
data

## Apply embedding layer

In [ ]:
simple_embed_layer(data[:, 2])

In [ ]:
embed_data_df = pd.DataFrame(torch.cat((data[:, :2].float(), simple_embed_layer(data[:, 2])), dim=1).numpy(), columns=['id', 'ts', 'E0', 'E1', 'E2'])
# Only use the line of code bellow if you want to test on Dask
# embed_data_df = dd.from_pandas(embed_data_df, npartitions=2)
# If using Pandas, uncomment the line of code bellow and comment the next one, which uses Dask
embed_data_df
# embed_data_df.compute()

## Apply embedding bag

In [ ]:
Var0_embed, Var0_offset = du.embedding.prepare_embed_bag(data_tnsr, features=ohe_feat)
print(f'Var0_embed: {Var0_embed}')
print(f'Var0_offset: {Var0_offset}')

In [ ]:
embedding_values = bag_embed_layer(Var0_embed, Var0_offset)[:-1]
embedding_values

In [ ]:
bag_embed_layer.embedding_dim

In [ ]:
data_tnsr.shape

In [ ]:
embedding_values.shape

In [ ]:
embedding_values.view(data_tnsr.shape[0], data_tnsr.shape[1], bag_embed_layer.embedding_dim)

In [ ]:
embedding_values_3d = embedding_values.view(data_tnsr.shape[0], data_tnsr.shape[1], bag_embed_layer.embedding_dim)
embedding_values_3d

In [ ]:
torch.cat((data.double(), embedding_values_3d.double()), dim=2)

Run the full embedding values calculation and tensor joining pipeline:

In [ ]:
data_df_ohe

In [ ]:
data_embed = du.embedding.embedding_bag_pipeline(data_tnsr, bag_embed_layer, features=ohe_feat)
data_embed

In [ ]:
data_embed = data_embed.view(-1, 5).numpy()
data_embed = data_embed[data_embed[:, 0] != 999999]
data_embed

In [ ]:
data_df_embed = pd.DataFrame(data_embed, columns=['id', 'ts', 'E0', 'E1', 'E2'])
data_df_embed

In [ ]:
data_df_embed.to_csv('dummy_data/embedding_bag_data_df_embed.csv')